In [188]:
import pandas as pd
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import math
from mpl_toolkits import mplot3d

color_list =["#3eb991","#e9a820","#e01563","#edb196","#6ecadc","#1f94ac","#ae9a6a","#ccb8a6","#343a44"]

In [189]:
# defining function for diffusion in 3D over time
def calculate_diffusion(total_time, delta_T):
    
    # defining variables
    v = 1 * pow(10, -6); 
    omega = 0; 
    D_r = 0.05; 
    D_t = 0.1 * pow(10, -12); 

    temp_X, temp_Y, delta_X, delta_Y = 0, 0, 0, 0; 
    start_X, start_Y = 0, 0; 

    temp_theta, delta_theta = 0, 0; 
    start_theta = 0; 

    # starting point defined as (start_X, start_Y, start_Z)
    X = np.array([start_X]); 
    Y = np.array([start_Y]); 
    theta = np.array([start_theta]); 

    # defining starting position of the particle
    temp_X = start_X; 
    temp_Y = start_Y; 
    temp_theta = start_theta; 

    for time in np.arange(0, total_time - delta_T, delta_T): 

        # randomly sampling in a uniform distribution from 0 to 1 to get an angle
        W_theta = (math.pi / 2) * random.uniform(-1, 1); 

        # randomly sampling noise in Gaussian distrib. with mean 0 and variance 1
        W_X = np.random.normal(0, 1); 
        W_Y = np.random.normal(0, 1); 

        # solving change in position for each dimension
        delta_X = v * np.cos(temp_theta) * delta_T + np.sqrt(2 * D_t * delta_T) * W_X; 
        delta_Y = v * np.sin(temp_theta) * delta_T + np.sqrt(2 * D_t * delta_T) * W_Y; 

        # solving change in each angle
        delta_theta = omega * delta_T + np.sqrt(2 * D_r * delta_T) * W_theta; 

        # using a temporary variable to track position and angle
        temp_X += delta_X; 
        temp_Y += delta_Y; 
        temp_theta += delta_theta; 

        # appending new position to arrays storing position over time
        X = np.append(X, [temp_X]); 
        Y = np.append(Y, [temp_Y]); 
    
    return X, Y, D_t; # returning coordinates and diffusion coefficient (for comparison later)

In [190]:
# defining function for diffusion in 3D over time
def single_step(X, Y, theta, delta_T):
    
    # defining variables
    v = 1 * pow(10, -6); 
    omega = 0; 
    D_r = 0.05; 
    D_t = 0.1 * pow(10, -12); 

    delta_X, delta_Y = 0, 0; 
    delta_theta = 0; 

    # randomly sampling in a uniform distribution from 0 to 1 to get an angle
    W_theta = (math.pi / 2) * random.uniform(-1, 1); 

    # randomly sampling noise in Gaussian distrib. with mean 0 and variance 1
    W_X = np.random.normal(0, 1); 
    W_Y = np.random.normal(0, 1); 

    # solving change in position for each dimension
    delta_X = v * np.cos(theta) * delta_T + np.sqrt(2 * D_t * delta_T) * W_X; 
    delta_Y = v * np.sin(theta) * delta_T + np.sqrt(2 * D_t * delta_T) * W_Y; 

    # solving change in each angle
    delta_theta = omega * delta_T + np.sqrt(2 * D_r * delta_T) * W_theta; 

    # using a temporary variable to track position and angle
    X += delta_X; 
    Y += delta_Y; 
    theta += delta_theta; 
    
    return X, Y, theta; # returning coordinates and diffusion coefficient (for comparison later)

In [191]:
def addnew(x): 
    X = [0]; 
    Y = [0]; 
    theta = [0]; 
    return 0; 

In [192]:
class makeSwimmer:
  def __init__(self, x, y, theta):
    self.x = x; 
    self.y = y; 
    self.theta = theta; 
    self.position = [x, y, theta]; 

In [193]:
swimmerList = []; 
Swimmers = [makeSwimmer(0, 0, 0) for i in range(10)]
for Swimmer in Swimmers:
    swimmerList.append(Swimmer); 

print(swimmerList[0]); 

In [194]:
total_time = 1; 
delta_T = 0.1; 
indexed_time = int(total_time / delta_T); 

X = [0]; 
Y = [0]; 
theta = [0]; 

evolution = np.zeros((3, 1, 1)); 
print(evolution.shape); 

temp_N = np.zeros((3, 1)); 
print(temp_N.shape); 

evolution = np.dstack((evolution, temp_N)); 
print(evolution.shape); 



counter = 0; 
for i in np.arange(0, total_time, delta_T):
    temp_X, temp_Y, temp_theta = single_step(X[counter], Y[counter], theta[counter], delta_T); 
    X = np.append(X, [temp_X]); 
    Y = np.append(Y, [temp_Y]); 
    theta = np.append(theta, [temp_theta]); 
    counter += 1; 






# 2D graphing definitions 
# fig, ax = plt.subplots()

# plt.tight_layout(); 

# plt.plot(X, Y, color_list[2]); # graphing line using color in colorlist (defined in the includes)

# plt.xlabel('x')
# plt.ylabel('y')
# ax.set_title("path")
# ax.set_aspect('equal', adjustable='box')
# plt.tight_layout()

(3, 1, 1)
(3, 1)
(3, 1, 2)
